In [ ]:
import pandas as pd
import numpy as np
from statsmodels.sandbox.regression.gmm import IV2SLS  # Instrumental Variable Regression
from statsmodels.stats.diagnostic import het_breuschpagan  # For exogeneity test

# Cargar la base de datos
base = pd.read_excel("base_p.xlsx")

# VARIABLES INDEPENDIENTES, INSTRUMENTALES Y CONTROLES
dependiente = "ptf"  # Productividad Total
instrumentales = ["D_calidad", "D_mercado", "D_exclusivo", "D_orden_compra", "D_exp_gerente"]  # IVs
controles = ["D_capacitacion", "D_inversion_maq_eq", "D_certificacion", "D_credito",
             "D_tec_1", "D_tec_2", "D_egrande", "D_emediano", "D_epequeño",
             "D_internet", "D_pagina_web", "L_promedio"]  # Controles

# Inicializar dataframe de resultados
resultados = []

# **PRUEBAS INDIVIDUALES**
for iv in instrumentales:
    # Modelo de IV2SLS (D_certificacion como endógena)
    model = IV2SLS(base[dependiente], base[controles + ["D_certificacion"]],
                   instrument=base[[iv]]).fit()

    # F-Test y t-Test
    f_test = model.fvalue
    t_test = model.tvalues[iv]

    # Test de Exogeneidad (Durbin-Wu-Hausman)
    resid = model.resid
    exog_test = het_breuschpagan(resid, base[controles + ["D_certificacion"]])
    pval_exog = exog_test[1]  # p-valor del test

    # Significancia
    signif_f = "***" if f_test > 10 else "**" if f_test > 5 else "*"
    signif_t = "***" if abs(t_test) > 2.58 else "**" if abs(t_test) > 1.96 else "*"
    signif_exog = "***" if pval_exog < 0.01 else "**" if pval_exog < 0.05 else "*" if pval_exog < 0.1 else ""

    # Almacenar resultados
    resultados.append([iv, f_test, t_test, pval_exog, signif_f, signif_t, signif_exog])

# **PRUEBAS CONJUNTAS**
combinaciones = [
    ("D_calidad", "D_mercado"), ("D_mercado", "D_exclusivo"),
    ("D_exclusivo", "D_orden_compra"), ("D_exp_gerente", "D_orden_compra")
]

for pair in combinaciones:
    # Modelo de IV2SLS para dos IVs
    model = IV2SLS(base[dependiente], base[controles + ["D_certificacion"]],
                   instrument=base[list(pair)]).fit()

    # F-Test y t-Test
    f_test = model.fvalue
    t_test = model.tvalues[pair[0]]  # Evaluar el primer instrumento de la combinación

    # Test de Exogeneidad
    resid = model.resid
    exog_test = het_breuschpagan(resid, base[controles + ["D_certificacion"]])
    pval_exog = exog_test[1]

    # Significancia
    signif_f = "***" if f_test > 10 else "**" if f_test > 5 else "*"
    signif_t = "***" if abs(t_test) > 2.58 else "**" if abs(t_test) > 1.96 else "*"
    signif_exog = "***" if pval_exog < 0.01 else "**" if pval_exog < 0.05 else "*" if pval_exog < 0.1 else ""

    # Almacenar resultados
    nombre_combinacion = f"{pair[0]} & {pair[1]}"
    resultados.append([nombre_combinacion, f_test, t_test, pval_exog, signif_f, signif_t, signif_exog])

# EXPORTAR RESULTADOS
df_resultados = pd.DataFrame(resultados, columns=["Variable(s)", "F-Test (Stock & Yogo)",
                                                  "t-Test (Lee)", "Exogeneity Test (p-value)",
                                                  "Signif. F-Test", "Signif. t-Test", "Signif. Exogeneity"])
df_resultados.to_excel("Resultados_IV.xlsx", index=False)
print("Los resultados han sido exportados a 'Resultados_IV.xlsx'")


MissingDataError: exog contains inf or nans